In [53]:
# Sau khi sử dụng K-NN với bài toán phân loại hoa 
# Ta tiến hành sử dụng kỹ thuật lõi là thuật toán K-NN cho bài toán 
# Cụ thể tên của phương pháp này là Neighborhood-based Collaborative Filtering (NBCF)

In [54]:
# Về mặt ý tưởng
# thì thuật toán này chính là sử dụng K-NN với .Với việc xác định mức độ quan tâm của một User với
# một item khác dựa trên K user gần đó nhất 
# Để xác định việc là các user có mực độ liên quan đến nhau như thế nào chúng ta có thể tính toán dựa trên lịch sử
# của chúng 
# Sẽ có 2 hướng tiếp cận dựa trên ý tưởng này là item-item (Mực độ giống nhau của các item ), (user-user) mức độ 
# giống nhau của các user 
# NHƯ VẬY :
# Với bài toán này ta cần quan tâm 2 điều 
# 1- Làm sao tính toán chuẩn sác được mức độ giống nhau giữa các item  ?
# 2 - Sau khi tính toán toán đc mức độ giống nhau , làm sao để xác định User đó có thích item này hay không ?

In [55]:
# DATA
# Ta sẽ sử dụng tập dữ liệu https://grouplens.org/datasets/movielens/100k/ có độ lớn là 5mb

In [56]:
# Vấn đề 1 - Tính toán sự giống nhau 
# Chúng ta có 1 ma trận của các user , và điểm được đánh cho các item mà user đó đánh 
# Cách xây dựng feature cho vector sẽ là u0(i1,i2......in), u1(i1, i2.....in) cứ như thế đến un(i1....in)
# Vấn đề gặp phải ở đây là : 
# Các user thướng rất ít khi đánh giá các item , thậm chí có nhiều cột toàn là giá trị N/A
# Để giải quyết vấn đề này , đầu tiên chúng ta sẽ tính giá trị trung bình của mà user đó vote cho các phim được xác 
# định trước đó 
# Sau khi có giá trị trung bình của các user
# Ta tiến hành trừ các giá trị rate trước đó cho giá trị trung bình => các giá trị chưa được điền sẽ có g trị là 0
# ĐIỀU NÀY GIẢI QUYẾT 2 VẤN ĐỀ 
# 1 : Sẽ có những User cho điểm rất cao - dễ tính , User cho điểm rất thấp - khó tính . việc trừ như vậy sẽ tìm ra được
#    mức độ hài lòng của user đó 
# 2 : Việc lưu trữ toàn bộ giá trị ma trận sẽ rất tốn kém bộ nhớ
#     Thay vào đó chúng ta sẽ chỉ lưu những ô có giá trị , còn những ô chưa được dự đoán sẽ ko mang giá trị 0 
#     Tức là chưa hề được đánh giá 

In [57]:
# SAU KHI CHUẨN HOÁ DỮ LIỆU => TÍNH TOÁN ĐỘ GIỐNG NHAU 
# Chúng ta có 2 công thức hay được sử dụng là Person và Cosine 
# Ở các bài dữ liệu nhỏ ta có thể dễ dàng nhìn ra các nhóm user có mối quan tâm giống nhau , tuy nhiên với dữ liệu
# lớn ta sẽ phải sử dụng thuật toán Users Clustering


In [58]:
# KNN và CF 
# CF sẽ là tính độ yêu thích của user với item đó dựa trên K phần tử * Đã rated item đó *
# Tuy nhiên điểm khác biệt là trọng số thay vì là khoảng cách như bài K-NN (1/distance) thì giờ đây nó là kết quả 
# của hàm cosine ( từ -1 đến 1 )
# Ta cần thêm dấu giá trị tuyệt đối 


In [62]:
#CODE 
# Import library and Algorithm in Sklearn
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

class CF(object):
    def __init__(self, Y_data, k, dist_func = cosine_similarity, uuCF=1):
        self.uuCF = uuCF # user-user (1) or item-item (0) CF
        self.Y_data = Y_data if uuCF else Y_data[:, [1, 0, 2]]
        self.k = k # number of neighbor points
        self.dist_func = dist_func # distance function
        self.Ybar_data = None
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1  # max of users
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1  # max of items 
        # Gia su o day id duoc danh thu tu 1-> max 
    
    def add(self, new_data):
        self.Y_data = np.concatenate((self.Y_data, new_data), axis =0) # add du lieu vao hang
    
    def normalize_Y(self):
        users = self.Y_data[:,0]
        self.Ybar_data = self.Y_data.copy()
        self.mu = np.zeros((self.n_users))
        for n in range(self.n_users):
            # duyệt các user theo danh sách users có được  
            ids = np.where(users == n)[0].astype(np.int32)
            # chuyển đổi các index xuất hiện của user hoặc item (hàng )            
            item_ids = self.Y_data[ids, 1]
            # danh sách các items mà user này vote 
            ratings = self.Y_data[ids, 2]
            # danh sách các điểm mà user này vote 
            m = np.mean(ratings) 
            # tính trung bình cộng của cái ratings này 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            # normalize
            self.mu[n]=m
            # Gán giá trị trung binh của từng user vào mảng của nó
            self.Ybar_data[ids, 2] = ratings - self.mu[n]

            #  Sử dụng ma trận thưa thớt - (mảng các cột - giá trị lưu tại mỗi hàng )
            # Lý do số lượng data có giá trị 0 - chưa vote là nhiều 
            self.Ybar = sparse.coo_matrix((self.Ybar_data[:, 2],
                (self.Ybar_data[:, 1], self.Ybar_data[:, 0])), (self.n_items, self.n_users))
            self.Ybar = self.Ybar.tocsr()
    
    def similarity(self):
        self.S = self.dist_func(self.Ybar.T, self.Ybar.T)
    
    def refresh(self):
        """
        Normalize data and calculate similarity matrix again (after
        some few ratings added)
        """
        self.normalize_Y()
        self.similarity() 
    
    def fit(self):
        self.refresh()
        
        
    def __pred(self, u, i, normalized = 1):
        """ 
         Đưa ra dự đoán điểm mà user u sẽ đánh giá cho item i
        """
        #print('Thứ nhất đang xét user %d và item %d' %(u,i))
        #print('Bước 1 : Tìm các index user đã voted cho item ',i)
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
        #print('Bước 2: Tìm các user đã voted cho item đó ', i)
        users_rated_i = (self.Y_data[ids, 0]).astype(np.int32)
        #print(users_rated_i)
        #print('Step 3: tính toán độ giống nhau giữa các user đó với user đang xét ')
        sim = self.S[u, users_rated_i]
        #print(sim)
        #print('Step 4: Tìm ra k thằng có mỗi liên quan nhất')
        a = np.argsort(sim)[-self.k:] 
        #print(a)
        #print('Đưa ra độ giống nhau của k thằng gần với user đang xét nhất  ')
        nearest_s = sim[a]
        #print(type(nearest_s))
        #print(nearest_s)
        #print('K users gần này đã vote item i', i)
        r = self.Ybar[i, users_rated_i[a]]
        #print(r)
        if normalized:
            # Thực hiện công thức tính toán rate tương đối của user u cho item i 
            return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8)

        return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8) + self.mu[u]
    
    
    def pred(self, u, i, normalized = 1):
        """ 
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        #print(self.__pred(u, i, normalized))
        if self.uuCF: return self.__pred(u, i, normalized)
        return self.__pred(i, u, normalized)

    
    def recommend(self, u, normalized = 1):
        """
        Determine all items should be recommended for user u. (uuCF =1)
        or all users who might have interest on item u (uuCF = 0)
        The decision is made based on all i such that:
        self.pred(u, i) > 0. Suppose we are considering items which 
        have not been rated by u yet. 
        """
        ids = np.where(self.Y_data[:, 0] == u)[0]
        items_rated_by_u = self.Y_data[ids, 1].tolist()              
        recommended_items = []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                rating = self.__pred(u, i)
                if rating > 0: 
                    print('diem rating cua user %d cho item %d la bao nhieu %f' %(u,i,rating))
                    recommended_items.append(i)
        
        return recommended_items 
    
    def print_recommendation(self):
        """
        print all items which should be recommended for each user 
        """
        print('Recommendation: ')
        for u in range(self.n_users):
            recommended_items = self.recommend(u)
            if self.uuCF:
                print('Recommend item(s): to user : ',recommended_items, u)
            else: 
                print ('Recommend item , to user(s): ', recommended_items)
            
            
                


In [64]:
'''
Chay thử với tập ex.dat tạo bằng tay
r_cols = ['user_id', 'item_id', 'rating']
ratings = pd.read_csv('ex.dat', sep = ' ', names = r_cols, encoding='latin-1')
Y_data = ratings.to_numpy()

rs = CF(Y_data, k = 2, uuCF = 1)
rs.fit()

rs.print_recommendation()

'''


print('Chay voi Data Movie lens 100k :')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.to_numpy()
rate_test = ratings_test.to_numpy()

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1
# run ..
rs = CF(rate_train, k = 10, uuCF = 1)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in range(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1],  normalized = 0)
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print('CF By User-user with RMSE:  =', RMSE)


Chay voi Data Movie lens 100k :
CF By User-user with RMSE:  = 1.005287691706029


Kết quả RMSE 1.000 là một kết quả không tốt cho lắm , ta thấy rằng đúng nư dự đoán . K-NN gặp vấn đề với lượng dữ liệu lớn , số điểm dữ liệu lớn.